## FAKE NEWS CLASSIFICATION

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [2]:
df = pd.read_csv('Data/train_LSTM.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.dropna()
df.shape

(18285, 5)

## Independent features

In [6]:
print(df.columns)
X = df.drop(columns='label',axis=1)
print(X.columns)

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')
Index(['id', 'title', 'author', 'text'], dtype='object')


## Dependent Feature

In [7]:
y = df['label']
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64


In [8]:
messages = X.copy()
messages.reset_index(inplace = True)

In [9]:
print(X.shape)
print(messages.shape)

(18285, 4)
(18285, 5)


In [10]:
messages.head()


,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [11]:
print(messages['title'][2])

Why the Truth Might Get You Fired


In [12]:
print(messages.shape)
print(len(messages))

(18285, 5)
18285


In [13]:
lemmatizer = WordNetLemmatizer()

In [14]:
corpus = []
for i in range (len(messages)):
    review = re.sub('[^a-zA-z]', ' ', messages['title'][i])
    review = review.lower()
    corpus.append(review)
    word_tokenized = nltk.tokenize.word_tokenize(corpus[i])
    lemmatized_corpus = [lemmatizer.lemmatize(words) for words in word_tokenized if words not in set(stopwords.words('english'))]
    corpus[i] = " ".join(lemmatized_corpus)

In [15]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic [ photo ]',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor 

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus_main = tokenizer.texts_to_sequences(corpus)

In [17]:
corpus_main

[[29, 526, 558, 249, 136, 81, 449, 1734, 3154, 6801],
 [760, 6, 7, 87, 31, 716, 5],
 [293, 784, 34, 695],
 [507, 132, 1082, 8, 1975, 3519],
 [1111, 31, 2635, 4568, 9426, 221, 31, 6802, 110, 9427],
 [3520, 3521, 294, 127, 322, 4, 3972, 102, 284, 1412, 2101, 1217, 209, 14, 5],
 [6803, 6804, 50, 337, 2414, 106, 130, 1218, 1, 3, 2],
 [59, 1489, 47, 1005, 16, 6805, 4, 1976, 1, 3, 2],
 [11, 6806, 527, 2240, 229, 965, 9428, 2636, 4, 785],
 [1112, 1846, 5433, 141, 5434, 77, 625, 1330],
 [48, 2241, 2415, 190, 966, 206, 967, 4569, 3155, 3156, 256],
 [8, 90, 136, 559, 4, 16],
 [1977, 733, 83, 1847, 413, 32, 1490, 4570, 968],
 [450, 1331, 1332, 5435, 88, 107, 1333, 1, 3, 2],
 [786, 1413, 155, 5436, 329, 1, 3, 2],
 [929, 677, 243, 60, 268, 930, 279, 346, 414, 280],
 [25, 374, 6],
 [1735, 1848, 2102, 9, 4, 147, 849, 5],
 [4571, 9429, 7, 172, 230, 236, 21, 191, 221],
 [4572, 9430, 4, 1978, 9431, 5],
 [560, 1979, 103, 393, 2637, 21, 244, 415, 787, 1, 3, 2],
 [2862, 138, 1414, 1736, 91, 179, 430],
 [20

In [18]:
print(len(corpus[2]))
print(len(corpus_main[2]))
print(max([len(i) for i in corpus_main]))
print([i for i in corpus_main if len(i) == 47])


21
4
47
[[888, 1615, 920, 145, 691, 1389, 161, 811, 2038, 666, 2973, 724, 4744, 374, 142, 477, 76, 161, 1071, 126, 323, 350, 4744, 7098, 1923, 173, 307, 2674, 3644, 229, 6, 1011, 2262, 8, 10, 7099, 22, 1689, 351, 502, 609, 21, 49, 563, 5629, 5630, 6896]]


In [19]:
## padding

corpus_main = pad_sequences(corpus_main, maxlen=10)

In [20]:
corpus_main

array([[  29,  526,  558, ..., 1734, 3154, 6801],
       [   0,    0,    0, ...,   31,  716,    5],
       [   0,    0,    0, ...,  784,   34,  695],
       ...,
       [6585,  348, 2352, ...,    1,    3,    2],
       [   0,    0,    0, ..., 4595, 2705, 5652],
       [   0,    0,    0, ...,  239,   96, 1410]], dtype=int32)

In [21]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus_main,y,test_size=0.3, random_state=32)

In [22]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(12799, 10)
(12799,)
(5486, 10)
(5486,)


In [23]:
model = Sequential()

In [24]:
len(tokenizer.word_index)

17487

In [25]:
embedding_layer = Embedding(input_length = 10 ,input_dim=tokenizer.word_index, output_dim= 10)

/opt/anaconda3/envs/tensor/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [26]:
model.add(embedding_layer)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['Accuracy'])

2024-07-20 21:00:16.402896: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-07-20 21:00:16.402932: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-07-20 21:00:16.402938: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-07-20 21:00:16.402975: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-20 21:00:16.402990: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
embedding.get_weights()[0]

IndexError: list index out of range